# Setup - Load an API Key and Libraries

## Storing and Loading Your API Key Securely

To securely store and use your API key in this notebook, follow these steps:

1. **What is a `.env` File?**  
   A `.env` file is a simple text file used to store sensitive information, such as API keys, in a secure and organized way. It allows you to keep sensitive data out of your code, making it easier to share or version-control your scripts without exposing private information. See our [guide](https://eodatahub.org.uk/docs/documentation/notebooks/sensitive-data/) for more information.

2. **Generate an API key**  
   Follow instructions in the [Getting Started documentation](https://eodatahub.org.uk/docs/documentation/apis/getting-started/) for Hub APIs to generate a Hub API Key.

3. **Create a `.env` File**
   - In the same directory as this notebook, create a plain text file named `.env`. You don't need any special tools or scripts - just create a regular file and name it .env.
   - Add the following line to the file, replacing `<your_api_key>` with your actual API key:
     ```plaintext
     API_KEY=<your_api_key>
     ```
   - Save the file.

4. **Load the Key in the Notebook**  
   The following code snippet is already included in this notebook to load the key securely:
   ```python
   import os
   from dotenv import load_dotenv
   
   load_dotenv(".env")  # Ensure the path matches your `.env` file location
   key = os.getenv("API_KEY")
   ```
   This will load the `API_KEY` from your `.env` file into the `key` variable.

5. **Keep the `.env` File Secure**
   - Do not share your `.env` file or API key.
   - If you do accidentally share your key, delete it in the workspaces UI and request a new one.

By following these steps, you can securely store and use your API key without exposing it in the notebook.

## Setting Up the Workspace and Environment

To ensure the notebook works correctly, you need to configure the `workspace` and `environment` variables. Follow these steps:
1. **Ensure your workspace is configured for commercial orders**
   - Follow the [Commercial Data](https://staging.eodatahub.org.uk/data/commercial/link-accounts/) guide to link your commercial account to your workspace.
2. **Set the `workspace` Variable**:
   - In the following cell, the `workspace` variable should be set to the name of the workspace you wish to order data for.
   - Check the [Workspaces page](https://eodatahub.org.uk/workspaces/) to view names of workspaces available to you.

> **Note for Developers**:  
> Most users do not need to change the `platform_domain` variable, as it is already set to the correct value for the production environment. However, developers with access to other platforms (e.g., staging, test, or dev environments) may need to update the `platform_domain` to match the appropriate environment. For example:  
> - Staging: `https://staging.eodatahub.org.uk`  
> - Test: `https://test.eodatahub.org.uk`  
> - Dev: `https://dev.eodatahub.org.uk`

In [ ]:
import os
from dotenv import load_dotenv

workspace = "my-workspace"

load_dotenv(".env")
api_key = os.getenv("API_KEY")

platform_domain = "https://eodatahub.org.uk"

In [ ]:
!pip install pystac-client xarray rasterio matplotlib

# Create a pystac-client Client for the EODH Catalogue

The EODH catalogue can be viewed through a UI on the Hub under the `Catalogue` tab.

Our STAC catalogue can also be browsed programatically using tools such as pystac-client, using your api_key for authorisation. In the following example, our client is scoped to the Planet catalogue to refine a search for some Planet commercial data.

In [ ]:
from pystac_client import Client

# Limit scope of the top-level catalogue to Planet
rc_url = f"{platform_domain}/api/catalogue/stac/catalogs/commercial/catalogs/planet"

# Create STAC client
stac_client = Client.open(rc_url, headers={"Authorization": f"Bearer {api_key}"})

# Search for Planet Data

To demonstrate search capabilities, the following cell searches our catalogue for PSScene items that intersect a given polygon. If you have already selected the imagery you would like to order, you can skip this step. Further examples of search capabilities are available in our [API tutorials](https://eodatahub.org.uk/docs/documentation/apis/example-tutorials/).

Coordinates are to be provided in longitude/latitude WGS84, and the polygon must be closed. Coordinates follow OGC GeoJSON specification. If you don’t have coordinates to hand, you can generate them using tools like [Google Earth](https://earth.google.com/) or any GIS software. Coordinates can define a polygon with more than 5 points inside the array, allowing for more complex shapes.

You can adjust the `max_items` variable to control the number of image results returned.

In [ ]:
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1],
        ]
    ],
}
search = stac_client.search(
    max_items=10,
    collections=['PSScene'],
    intersects=geom,
)
for item in search.items():
    print(item.get_self_href())

# Obtain a Quote for Planet Data

Once an item of interest is found, you may obtain a quote for the item via a POST request to `/quote` following the `item_href`.

### How to Obtain the `item_href`
- The `item_href` can be obtained from the output of a search, such as the cell above, where the `get_self_href()` method returns the link.
- Alternatively, you can find the `item_href` in the Map user interface under the "Additional Resources" section for an item, labeled as the "self" link.

### Understanding the `coordinates` Field

The `coordinates` field is optional and can be included to specify an Area of Interest (AOI). If provided, the item will be clipped to the specified AOI, and the quote (and later purchase) will reflect only the intersection of the original item and your AOI. This can help reduce the area and cost of the data you are requesting.

If the `coordinates` field is omitted, the quote (and purchase) will be for the entire item.

Coordinates must follow the OGC GeoJSON specification and be provided in longitude/latitude WGS84. The polygon must be closed, meaning the first and last points in the array should be the same. For example:

```json
"coordinates": [
    [
        [9.6, 57.1],
        [9.6, 57.0],
        [9.8, 56.9],
        [9.8, 57.0],
        [9.6, 57.1]
    ]
]
```
You can generate these coordinates using tools like [Google Earth](https://earth.google.com/) or any GIS software.

### Understanding the Response
- The `response` from the POST request is a JSON output containing the amount and unit of the quote. For Planet data, the unit is typically in square kilometers (km²).
- The quote is provided in square kilometers (km²) and represents an approximation of the area of data contained in the purchase.
- This area will count against your quota in the Planet system, which you can view on your [Planet account](https://docs.planet.com/platform/get-started/access-data/).

### Status Codes and Their Meaning
- **200 (OK)**: The request was successful, and the quote is returned in the response.
- **400 (Bad Request)**: The request is malformed. Check the input data `coordinates` field is valid if included.
- **401 (Unauthorized)**: The API key is missing or invalid. Ensure your `.env` file contains the correct API key.
- **404 (Not Found)**: The `item_href` is incorrect or invalid. Verify that the link is valid and corresponds to an existing item.
- **405 (Method Not Allowed)**: The HTTP method used is not supported for the endpoint. Ensure you are using a POST request.
- **500 (Internal Server Error)**: An issue occurred on the server. Retry the request later or contact support if the issue persists.

### Important Note
- You will **not** be charged for the quote itself. Charges will only apply if you proceed to place an order in a later cell.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/planet/collections/PSScene/items/20250217_101155_07_24c7"
url = f"{item_href}/quote"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

# Order Planet Data

Once you are satisfied with the item and pricing, you may proceed with ordering the item via a POST request to `/order` following the `item_href`.

- Change the `item_href` to any valid link to a STAC item in our Planet commercial catalogue to order it.  
- The `coordinates` argument is optional. If included, the ordered image will be clipped to the specified AOI. If omitted, the entire image will be purchased.  
- The `productBundle` field may also be changed to adjust the processing done on the image. For details, please see our [guide on placing orders](https://eodatahub.org.uk/data/commercial/ordering-commercial-data/).

Once an order is placed, the API response includes a `Location` header that links to the item in your workspace catalogue. This item will be updated with the order status and assets when they are ready.

> **Warning**: The lines of code that place the order and process the response are commented out by default. If you uncomment and run them with the correct setup, it will charge quota to your account. Ensure you review all parameters carefully before proceeding.

An order cannot be changed once it is placed, so please ensure the correct product and parameters are chosen.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/planet/collections/PSScene/items/20250217_101155_07_24c7"
url = f"{item_href}/order"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "productBundle": "General Use",
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1]
        ]
    ]
}

# Uncomment the following lines to place an order

# response = requests.post(url, headers=headers, json=data)

# print("Status Code:", response.status_code)
# print("Response:", response.json())

# location_header = response.headers.get('Location')
# print("Item will shortly be available at:", location_header)

# Read STAC for Ordered Planet Data

The STAC item can also be obtained through browsing your workspace catalogue. The following example demonstrates a search for a specific asset for an item.

In [ ]:
from pystac import Item
from pystac_client import CollectionClient

data_i_ordered_earlier = f"{platform_domain}/api/catalogue/stac/catalogs/user/catalogs/{workspace}/catalogs/commercial-data/catalogs/planet"
stac_client = Client.open(data_i_ordered_earlier, headers={"Authorization": f"Bearer {api_key}"})

In [ ]:
ordered_item = next(stac_client.get_items("20250217_101155_07_24c7"))
ordered_item

In [ ]:
asset = ordered_item.get_assets()["20250217_101155_07_24c7_3B_AnalyticMS_clip.tif"]
asset

# Fetch Ordered Planet Data

Any asset link from a commercial data purchase can be downloaded via an authorised GET request to the asset href.

In [ ]:
import urllib3
from io import BytesIO

resp = urllib3.request("GET", asset.href, headers={"Authorization": f"Bearer {api_key}"})
resp.data[0:100]

# Plot Planet Data

Visual assets can be viewed programmatically:

In [ ]:
from rasterio.plot import show
import rasterio
import matplotlib.pyplot as plt

with rasterio.open(BytesIO(resp.data)) as src:
    f = src.read()
    fig, ax = plt.subplots(figsize=(10, 10))
    show(f / 10000, ax=ax, title="PlanetScope Image")
    plt.show()

# Further Visualisation of Planet Data

Our TiTiler instance can also be used to visualise the data:

In [ ]:
import requests
from IPython.display import Image, display

TITILER_PREVIEW_URL = f'{platform_domain}/titiler/core/cog/preview'
TITILER_PREVIEW_PARAMS = {
    'url': asset.href,
    'bidx': 1,
    'rescale': '2229,12331',
    'colormap_name': 'rain_r'
}

response = requests.get(TITILER_PREVIEW_URL, params=TITILER_PREVIEW_PARAMS, headers={'Authorization': f'Bearer {api_key}'})

# Display the image
image = Image(response.content)
display(image)

# Obtain a Quote for Airbus SAR Data

Search and ordering capabilities are also in place for Airbus SAR items. Once an item of interest is found, you may obtain a quote for the item via a POST request to `/quote` following the `item_href`.

Note: Airbus SAR does not support clipping an item with an AOI.

### How to Obtain the `item_href`
- The `item_href` can be obtained from the output of a search, demonstrated earlier in this notebook for Planet items.
- Alternatively, you can find the `item_href` in the Map user interface under the "Additional Resources" section for an item, labeled as the "self" link.

### Understanding the `licence` field

The `licence` field must be populated for Airbus SAR quotes. It should be changed depending on how you intend to use the data. For more information see the [Airbus guide](https://space-solutions.airbus.com/legal/licences/). Supported licences are:
- `Single User Licence`
- `Multi User (2 - 5) Licence`
- `Multi User (6 - 30) Licence`.

### Understanding the Response
- The `response` from the POST request is a JSON output containing the amount and unit of the quote. For Airbus data, the unit is typically in Euros (EUR).
- The quote represents a monetary value that will be charged to your Airbus account upon placing the order.

### Status Codes and Their Meaning
- **200 (OK)**: The request was successful, and the quote is returned in the response.
- **400 (Bad Request)**: The request is malformed. Check the input data includes only a valid `licence`.
- **401 (Unauthorized)**: The API key is missing or invalid. Ensure your `.env` file contains the correct API key.
- **404 (Not Found)**: The `item_href` is incorrect or invalid. Verify that the link is valid and corresponds to an existing item.
- **405 (Method Not Allowed)**: The HTTP method used is not supported for the endpoint. Ensure you are using a POST request.
- **500 (Internal Server Error)**: An issue occurred on the server. Retry the request later or contact support if the issue persists.

### Important Note
- You will **not** be charged for the quote itself. Charges will only apply if you proceed to place an order in a later cell.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_WS_S_wide_001R_97985_D33003943_29000"
url = f"{item_href}/quote"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "licence": "Single User Licence",
}

response = requests.post(url, headers=headers, json=data)

print("Status Code:", response.status_code)
print("Response:", response.json())

# Order Airbus SAR Data

Once you are satisfied with the item and pricing, you may proceed with ordering the item via a POST request to `/order` following the `item_href`.

- Change the `item_href` to any valid link to a STAC item in our Airbus commercial catalogue to order it.
- `licence` must be set depending on the intended usage of the data. For more information see the [Airbus guide](https://space-solutions.airbus.com/legal/licences/). Supported licences are:
  - `Single User Licence`
  - `Multi User (2 - 5) Licence`
  - `Multi User (6 - 30) Licence`
- The `productBundle` and `radarOptions` fields may also be changed to adjust the processing done on the image. Valid product bundles for SAR are `SSC`, `MGD`, `GEC`, and `EEC`. `radarOptions` include:
  - `orbit` of `rapid` or `science`.
  - `resolutionVariant` of `RE` or `SE`.
  - `projection` of `Auto`, `UTM`, or `UPS`.
  
For further details please see our [guide on placing orders](https://eodatahub.org.uk/data/commercial/ordering-commercial-data/).

Once an order is placed, the API response includes a `Location` header that links to the item in your workspace catalogue. This item will be updated with the order status and assets when they are ready.

> **Warning**: The lines of code that place the order and process the response are commented out by default. If you uncomment and run them with the correct setup, it will charge quota to your account. Ensure you review all parameters carefully before proceeding.

An order cannot be changed once it is placed, so please ensure the correct product and parameters are chosen.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_WS_S_wide_001R_97985_D33003943_29000"
url = f"{item_href}/order"
headers = {
    "accept": "application/json", 
    "Content-Type": "application/json", 
    "Authorization": f"Bearer {api_key}"
}
data =  {
    "licence": "Single User Licence",
    "productBundle": "SSC",
    "radarOptions": {
        "orbit": "rapid"
    }
}

# Uncomment the following lines to place an order

# response = requests.post(url, headers=headers, json=data)

# print("Status Code", response.status_code)
# print("Response ", response.json())

# location_header = response.headers.get('Location')
# print("Item will shortly be available at:", location_header)

# Obtain a Quote for Airbus Optical Data

Search and ordering capabilities are also in place for Airbus Optical items. Once an item of interest is found, you may obtain a quote for the item via a POST request to `/quote` following the `item_href`.

### How to Obtain the `item_href`
- The `item_href` can be obtained from the output of a search, such as the cell above, where the `get_self_href()` method returns the link.
- Alternatively, you can find the `item_href` in the Map user interface under the "Additional Resources" section for an item, labeled as the "self" link.

### Understanding the `licence` field

The `licence` field must be populated for Airbus Optical quotes. It should be changed depending on how you intend to use the data. For more information see the [Airbus guide](https://space-solutions.airbus.com/legal/licences/). Supported licences are:
- `Standard`
- `Background Layer`
- `Standard + Background Layer`
- `Academic`
- `Media Licence`
- `Standard Multi End-Users (2-5)`
- `Standard Multi End-Users (6-10)`
- `Standard Multi End-Users (11-30)`
- `Standard Multi End-Users (>30)`

### Understanding the `coordinates` Field

The `coordinates` field is optional and can be included to specify an AOI. If provided, the item will be clipped to the specified AOI, and the quote (and later purchase) will reflect only the intersection of the original item and your AOI. This can help reduce the area and cost of the data you are requesting.

If the `coordinates` field is omitted, the quote (and purchase) will be for the entire item.

Coordinates must follow the OGC GeoJSON specification and be provided in longitude/latitude WGS84. The polygon must be closed, meaning the first and last points in the array should be the same. For example:

```json
"coordinates": [
    [
        [9.6, 57.1],
        [9.6, 57.0],
        [9.8, 56.9],
        [9.8, 57.0],
        [9.6, 57.1]
    ]
]
```
You can generate these coordinates using tools like [Google Earth](https://earth.google.com/) or any GIS software.

### Understanding the Response
- The `response` from the POST request is a JSON output containing the amount and unit of the quote. For Airbus data, the unit is typically in Euros (EUR).
- The quote represents a monetary value that will be charged to your Airbus account upon placing the order.

### Status Codes and Their Meaning
- **200 (OK)**: The request was successful, and the quote is returned in the response.
- **400 (Bad Request)**: The request is malformed. Check the input data includes a valid `licence` and the `coordinates` field is valid if included.
- **401 (Unauthorized)**: The API key is missing or invalid. Ensure your `.env` file contains the correct API key.
- **404 (Not Found)**: The `item_href` is incorrect or invalid. Verify that the link is valid and corresponds to an existing item.
- **405 (Method Not Allowed)**: The HTTP method used is not supported for the endpoint. Ensure you are using a POST request.
- **500 (Internal Server Error)**: An issue occurred on the server. Retry the request later or contact support if the issue persists.

### Important Note
- You will **not** be charged for the quote itself. Charges will only apply if you proceed to place an order in a later cell.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_phr_data/items/DS_PHR1A_201203021558128_FR1_PX_W080S03_0221_01728"
url = f"{item_href}/quote"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "licence": "Standard",
    "coordinates": [
        [
            [-79.8,-2.1], 
            [-79.8,-2.2], 
            [-79.95,-2.2], 
            [-79.95,-2.1], 
            [-79.8,-2.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

# Order Airbus Optical Data

Once you are satisfied with the item and pricing, you may proceed with ordering the item via a POST request to `/order` following the `item_href`.

- Change the `item_href` to any valid link to a STAC item in our Airbus commercial catalogue to order it. 
- `coordinates` can optionally clip an item to an AOI.
- `licence` must be set depending on the intended usage of the data. For more information see the [Airbus guide](https://space-solutions.airbus.com/legal/licences/). Supported licences are:
  - `Standard`
  - `Background Layer`
  - `Standard + Background Layer`
  - `Academic`
  - `Media Licence`
  - `Standard Multi End-Users (2-5)`
  - `Standard Multi End-Users (6-10)`
  - `Standard Multi End-Users (11-30)`
  - `Standard Multi End-Users (>30)`
- Set the `productBundle` field to adjust the processing done on the image. For details please see our [guide on placing orders](https://eodatahub.org.uk/data/commercial/ordering-commercial-data/).
- `endUserCountry` must match an Airbus country code corresponding to the user making the purchase.

Once an order is placed, the API response includes a Location header that links to the item in your workspace catalogue. This item will be updated with the order status and assets when they are ready.

> **Warning**: The lines of code that place the order and process the response are commented out by default. If you uncomment and run them with the correct setup, it will charge quota to your account. Ensure you review all parameters carefully before proceeding.

An order cannot be changed once it is placed, so please ensure the correct product and parameters are chosen.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_phr_data/items/DS_PHR1A_201203021558128_FR1_PX_W080S03_0221_01728"
url = f"{item_href}/order"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "licence": "Standard",
    "endUserCountry": "GB",
    "productBundle": "General Use", 
    "coordinates": [
        [
            [-79.8,-2.1], 
            [-79.8,-2.2], 
            [-79.95,-2.2], 
            [-79.95,-2.1], 
            [-79.8,-2.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code:", response.status_code)
print("Response:", response.json())

location_header = response.headers.get('Location')
print("Item will shortly be available at:", location_header)